In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import datetime
from progressbar import *
from IPython.display import clear_output

# Makalelerin Linklerinin Çekilmesi

In [ ]:
sections = ["https://www.dunyahalleri.com/category/haftanin-ozeti/page/",
            "https://www.dunyahalleri.com/category/genel-gundem/page/",
            "https://www.dunyahalleri.com/category/teknoloji-bilim/page/",
            "https://www.dunyahalleri.com/category/internet-girisimler/page/",
            "https://www.dunyahalleri.com/category/tasarim-inovasyon/page/",
            "https://www.dunyahalleri.com/category/kultur-sanat/page/"]

In [ ]:
urls = []
#Öncelikle bir Kategori seçiyoruz.
for section in sections:
    #Kategorinin içerisinde sırayla 100 sayfa gezineceğiz.
    for i in range(1,100):
        try:
            #Öncelikle URL'imizi oluşturuyoruz. Örneğin;
            #https://www.dunyahalleri.com/category/kultur-sanat/page/25
            newurl = section+str(i)
            print(newurl)

            #Url'nin içerisindeki bütün html dosyasını indiriyoruz.
            html = requests.get(newurl).text
            soup = bs(html, "lxml")

            #Yukarıdaki şemadada görüldüğü gibi bütün makaleler bu element'in içerisinde yer alıyor.
            #Bizde bütün makaleleri buradan tags adında bir değişkene topluyoruz.
            tags = soup.findAll("div", class_="row row-eq-height herald-posts")[0]

            #Sırayla bütün makalelere girip, href'in içerisindeki linki urls adlı listemize append ediyoruz.
            for a in tags.find_all('a', href=True):
                urls.append((section.split("/")[4],a['href']))
        except IndexError:
            break

https://www.dunyahalleri.com/category/haftanin-ozeti/page/1
https://www.dunyahalleri.com/category/haftanin-ozeti/page/2
https://www.dunyahalleri.com/category/haftanin-ozeti/page/3
https://www.dunyahalleri.com/category/haftanin-ozeti/page/4
https://www.dunyahalleri.com/category/haftanin-ozeti/page/5
https://www.dunyahalleri.com/category/haftanin-ozeti/page/6
https://www.dunyahalleri.com/category/haftanin-ozeti/page/7
https://www.dunyahalleri.com/category/haftanin-ozeti/page/8
https://www.dunyahalleri.com/category/haftanin-ozeti/page/9
https://www.dunyahalleri.com/category/haftanin-ozeti/page/10
https://www.dunyahalleri.com/category/haftanin-ozeti/page/11
https://www.dunyahalleri.com/category/haftanin-ozeti/page/12
https://www.dunyahalleri.com/category/haftanin-ozeti/page/13
https://www.dunyahalleri.com/category/haftanin-ozeti/page/14
https://www.dunyahalleri.com/category/haftanin-ozeti/page/15
https://www.dunyahalleri.com/category/haftanin-ozeti/page/16
https://www.dunyahalleri.com/cate

In [ ]:
urldata = pd.DataFrame(urls)
urldata.columns = ["Kategori","Link"]
urldata.head()

,Kategori,Link
0,haftanin-ozeti,https://www.dunyahalleri.com/yeniden-merhaba/
1,haftanin-ozeti,https://www.dunyahalleri.com/category/genel-gu...
2,haftanin-ozeti,https://www.dunyahalleri.com/yeniden-merhaba/
3,haftanin-ozeti,https://www.dunyahalleri.com/author/dunya-hall...
4,haftanin-ozeti,https://www.dunyahalleri.com/yeniden-merhaba/


In [ ]:
urldata = urldata.drop_duplicates()

In [ ]:
urldata.to_csv('urldata.csv')

# Linklerdeki Verilerin Çekilmesi

In [ ]:
def GetData(url):
    try:
        #Url içerisindeki html'i indiriyoruz.
        html = requests.get(url).text

        soup = bs(html, "lxml")

        #Belirlediğimiz element'in altındaki bütün p'leri seçiyoruz.
        body_text = soup.findAll("div", class_="entry-content herald-entry-content")[0].findAll('p')

        #Body_text adındaki metni tek bir string üzerinde topluyoruz.
        body_text_big = ""
        for i in body_text:
            body_text_big = body_text_big +i.text

        #Başlığı ve zamanı'da element isimlerinden bu şekilde seçip, metinlerini alıyoruz.
        header = soup.find("h1", class_="entry-title h1").text
        timestamp = soup.find("span", class_="updated").text

        #Özetin bulunduğu element'in metin kısmını alıyoruz.
        summarized = soup.find("div", class_="tldr-summary").text
        return ((url,header,body_text_big,summarized,timestamp))

    #Link boş ise verilen hata üzerine Boş Data mesajını dönüyor.
    except IndexError:
        return ("Boş Data")

    #Eğer link haftalık özet ise özet kısmı olmadığından oraya haftalık özet yazıp, sonuçlar o şekilde dönüyor.
    except AttributeError:
        return ((url,header,body_text_big,"Haftalık Özet",timestamp))


In [ ]:
bigdata = []
k = 0
for i in urldata.Link:
    print(i)
    clear_output(wait=True)
    print(k)
    print(GetData(i))
    bigdata.append(GetData(i))
    k = k + 1
    if k == 1000:
      break

999
('https://www.dunyahalleri.com/dertlesecek-erkek-kiralama-hizmeti/', '‘Dertleşecek erkek’ kiralama hizmeti', "Haber ÖzetiTam SürümJapon girişimci Tokanobu Nishimoto ve kendisi gibi orta yaşlı erkeklerden oluşan ekibi, para karşılığında müşterilerinin dertlerini dinliyor. Japoncada ‘ossan’ olarak adlandırılan orta yaşlı erkekler, insanların terapistlere ya da ailelerine anlatamadığı problemleri dinleyerek onlara yardımcı oluyor. Görüşmeler saati yaklaşık 30 TL\xa0karşılığında\xa0yapılıyor.Nishimoto bu hizmetin kendisi için her şeyden çok bir hobi olduğunu söylüyor. Bu girişimin çıkış noktası ise orta yaşlı erkeklerin imajını düzeltmek ve ciddiye alınmalarını sağlamak. Ekipte 60’ın üzerinde çalışan bulunuyor ve bu rakam sürekli\xa0artıyor.48 yaşında profesyonel bir moda yöneticisi olan Nishimoto, kendisini kiralayanlara sadece sohbet hizmeti sunduğunun altını çiziyor. Her ay 30 ila 40 arası müşteri Nishimoto ile görüşüyor. Bu müşterilerin yüzde 70’i kadınlardan oluşuyor.Müşterilerin 

In [ ]:
bigdatax = pd.DataFrame(bigdata)
bigdatax

,0,1,2,3,4,5,6,7
0,https://www.dunyahalleri.com/yeniden-merhaba/,Yeniden merhaba!,Küresel trendleri belirleyen gelişmeler hakkın...,Haftalık Özet,12/09/2021,None,None,None
1,B,o,ş,,D,a,t,a
2,B,o,ş,,D,a,t,a
3,https://www.dunyahalleri.com/hoscakalin/,Hoşçakalın,Dünya Halleri yayın hayatına 2 Kasım 2014 tari...,Haftalık Özet,06/08/2019,None,None,None
4,B,o,ş,,D,a,t,a
...,...,...,...,...,...,...,...,...
995,https://www.dunyahalleri.com/milenyum-gencleri...,Milenyum gençleri sevişmiyor,Haber ÖzetiTam SürümABD’de gençlerin cinsel ak...,Haftalık Özet,09/08/2016,None,None,None
996,https://www.dunyahalleri.com/yeni-araba-kokusu...,Yeni araba kokusu nereden gelir?,Haber ÖzetiTam SürümYeni araba kokusuna sebep ...,Haftalık Özet,06/08/2016,None,None,None
997,https://www.dunyahalleri.com/ceo-kazandikca-si...,CEO kazandıkça şirketi kaybediyor,Haber ÖzetiTam SürümKurumsal araştırma şirketi...,Haftalık Özet,29/07/2016,None,None,None
998,https://www.dunyahalleri.com/en-uzun-kisa-boyl...,En uzun ve kısa boylu milletler açıklandı,Haber ÖzetiTam SürümYeni yapılan bir araştırma...,Haftalık Özet,29/07/2016,None,None,None


In [ ]:

bigdatax.drop([5,6,7],axis=1,inplace=True)
bigdatax.drop(bigdatax[bigdatax[0]=="B"].index,axis=0,inplace=True)
bigdatax.columns = ["Link","Başlık","Body_text","Summarized_Text","TimeStamp"]
bigdatax = bigdatax.loc[bigdatax.Link.drop_duplicates().index]
bigdatax.index = range(0,len(bigdatax))
bigdatax.head()

,Link,Başlık,Body_text,Summarized_Text,TimeStamp
0,https://www.dunyahalleri.com/yeniden-merhaba/,Yeniden merhaba!,Küresel trendleri belirleyen gelişmeler hakkın...,Haftalık Özet,12/09/2021
1,https://www.dunyahalleri.com/hoscakalin/,Hoşçakalın,Dünya Halleri yayın hayatına 2 Kasım 2014 tari...,Haftalık Özet,06/08/2019
2,https://www.dunyahalleri.com/haftanin-spor-oze...,Haftanın Spor Özeti: 59,"Haftanın Spor Özeti, Dünya Halleri özetlerinin...",Haftalık Özet,05/08/2019
3,https://www.dunyahalleri.com/haftanin-ozeti-249/,Haftanın Özeti: 249,Dünyada bir yanda açlık çeken insanlar diğer y...,Haftalık Özet,04/08/2019
4,https://www.dunyahalleri.com/dizel-araclari-el...,Dizel araçları elektrikliye dönüştüren girişim,Haber ÖzetiTam SürümFransız girişimi Transitio...,Haftalık Özet,03/08/2019


In [ ]:
def Düzeltici(x):
    if x[:20] == "Haber ÖzetiTam Sürüm":
        return x[20:]
    else:
        return x

In [ ]:
bigdatax.Body_text = bigdatax.Body_text.apply(Düzeltici)

In [ ]:
droplist =[]
for i in range(0,len(bigdatax)):
    if bigdatax.loc[i].Body_text[:35] == 'Türkiye ve dünyadan güncel haberler':
        droplist.append(i)
    else:
        continue

In [ ]:
bigdatax.drop(droplist,axis=0,inplace=True)

In [ ]:
bigdatax.to_excel("DunyaHalleri.xlsx", encoding="utf-16")

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
